In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.font_manager as fm
import warnings

warnings.simplefilter("ignore")

In [ ]:
!pip install lightfm # 하이브리드 추천 시스템을 위한 대표 라이브러리
!pip install tensorflow-recommenders # 딥러닝 기반 추천 시스템을 위한 라이브러리

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831157 sha256=7a2b09f5b8244856b06fe3a66a2d9cd436a6c9f7f661d91c9bdcf1a0db56902f
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.2 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
from scipy.sparse import csr_matrix
from lightfm import LightFM
from lightfm.data import Dataset
from scipy.sparse import coo_matrix, csr_matrix


In [ ]:
# 1. 유저 취업 관련 정보와 채용 공고 텍스트
user_text = "저는 파이썬과 데이터 분석 경험이 있으며, 머신러닝 및 자연어 처리 프로젝트에 참여한 경험이 있습니다."
item_texts = [
    "백엔드 개발자 - Python, Django, REST API",
    "데이터 분석가 - SQL, Python, Tableau",
    "프론트엔드 개발자 - React, JavaScript, UI/UX",
    "AI 연구원 - 딥러닝, PyTorch, NLP",
    "DevOps 엔지니어 - AWS, Docker, CI/CD"
]

# 2. SBERT로 임베딩 (두 텍스트 모두 동일 모델 사용)
sbert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# 유저 텍스트 임베딩 (결과 shape: (1, embedding_dim))
user_embedding = sbert_model.encode([user_text], normalize_embeddings=True)
# 채용 공고 텍스트 임베딩 (결과 shape: (n_items, embedding_dim))
item_embeddings = sbert_model.encode(item_texts, normalize_embeddings=True)

# 3. LightFM이 요구하는 sparse matrix 형태로 변환
user_features = csr_matrix(user_embedding)
item_features = csr_matrix(item_embeddings)

# 4. Dataset 생성
# 유저는 한 명, 아이템은 채용 공고 리스트의 인덱스로 사용
dataset = Dataset()
dataset.fit(users=['user_1'], items=list(range(len(item_texts))))

# 5. Dummy interaction 생성
# 상호작용 데이터가 전무하면 모델 학습이 어려우므로,
# 여기서는 임의로 유저가 첫 번째 공고에 대해 긍정적 상호작용을 가졌다고 가정함.
interaction_data = [('user_1', 0), ('user_1', 1)]
(interactions, _) = dataset.build_interactions(interaction_data)

# 6. LightFM 모델 학습 (user_features와 item_features 모두 사용)
model = LightFM(loss='warp')
model.fit(interactions, user_features=user_features, item_features=item_features,
          epochs=30, num_threads=2)

# 7. 유저에 대해 모든 공고 점수 예측 후 추천 순위 도출
scores = model.predict(0, np.arange(len(item_texts)), user_features=user_features, item_features=item_features)
recommended_order = np.argsort(-scores)

print("추천 결과:")
for idx in recommended_order:
    print(f"{idx}: {item_texts[idx]} (score: {scores[idx]:.4f})")

추천 결과:
0: 백엔드 개발자 - Python, Django, REST API (score: 0.2264)
1: 데이터 분석가 - SQL, Python, Tableau (score: 0.1611)
3: AI 연구원 - 딥러닝, PyTorch, NLP (score: -0.6313)
2: 프론트엔드 개발자 - React, JavaScript, UI/UX (score: -0.6315)
4: DevOps 엔지니어 - AWS, Docker, CI/CD (score: -0.8717)
